In [2]:
from subscription.enum import *
from backend.stripe import stripe

import itertools

In [3]:
user = User.objects.get(email='veyorokon@gmail.com')

In [3]:
customer = Customer.objects.get(id='cus_Ely4ei6DSDgze2')
customer.subscriber